# Extract Drawing ROI
This notebook extracts the coordinates of the drawing using the output of the `fastroi` processing. The output is a CSV file with each image path and the ROI of the drawing within the image. The ROI is computed given the estimated XY center by setting a box around it of 1.5x the size of the template height and width.

In [ ]:
import sys
import math
from pathlib import Path
from typing import Dict, List

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from bananas.utils import images

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Extract the coordinates from the individual CSV files in the `fastroi` folder

In [ ]:
def coords_to_roi(coords: List[int], tpl: Template, img_size: List[int], scaling: float = 1.5):
    ''' Converts XY center coordinates into a box assuming similar size compared to template'''
    
    # Open image to determine height and width
    y_max, x_max = img_size[0], img_size[1]
    
    # Estimate the halves of the resulting ROI in whole pixels
    w_max_half = int(tpl.width * scaling / 2)
    h_max_half = int(tpl.height * scaling / 2)
    
    return Box(
        left = max(0, coords[0] - w_max_half),
        top = max(0, coords[1] - h_max_half),
        right = min(x_max, coords[0] + w_max_half),
        bottom = min(y_max, coords[1] + h_max_half))

In [ ]:
# Iterate over all drawing ROIs
drawing_roi_records = []
for line in tqdm(open(ROOT / 'datasets' / 'fastroi_output.csv').readlines()):
    impath, coords = line.split(',', 1)
    coords = list(map(int, coords.split(',')))

    # Open the image and extract its dimensions
    img_shape = images.open_image(str(ROOT / impath)).shape

    # Derive template from path and extract approximate coordinates
    tpl = tpl_load(ROOT / 'templates' / (impath.split('/')[1].lower() + '.png'))
    drawing_roi = coords_to_roi(coords, tpl, img_shape)

    drawing_roi_records.append({
        'image_path': impath,
        'drawing_box': str(drawing_roi)})

print('\nROIs processed: %d' % len(drawing_roi_records))

### Write results in a CSV file

In [ ]:
pd.DataFrame.from_records(drawing_roi_records).set_index('image_path').to_csv(ROOT / 'datasets' / 'drawing_roi.csv')